# 0. Import standard packages

In [175]:
import pandas as pd 
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Set-Up of Matrices for Collaborative filtering

First we want to implement the ratings matrix from Table 2.1 featuring 5 users and 6 items. We want to employ a simple user-based and item-based model for imputing ratings. For that we will use the cosine and the pearson-correlation-coefficient.

In [176]:
# create the tables 2.1 and 2.2

table_2_1 = np.array([[7, 6, 7, 4, 5 , 4], [6,7,np.nan, 4,3,4], [np.nan, 3, 3, 1, 1, np.nan], [1, 2, 2 , 3 ,3 , 4], [1, np.nan, 1, 2, 3, 3]])

In [177]:
table_2_1

array([[ 7.,  6.,  7.,  4.,  5.,  4.],
       [ 6.,  7., nan,  4.,  3.,  4.],
       [nan,  3.,  3.,  1.,  1., nan],
       [ 1.,  2.,  2.,  3.,  3.,  4.],
       [ 1., nan,  1.,  2.,  3.,  3.]])

Next, we want to compute the pearson-correlation-coefficient. The formula for this is:

$ \mu_u = \frac{\sum_{k \in I_{u}} r_{uk}}{|I_{u}|} $

$  Sim(u,v) = Pearson(u,v) = \frac{\sum_{k \in I_{u} \cap I_{v}} (r_{uk}- \mu_{u}) \cdot (r_{uk}- \mu_{u}) }{\sqrt{\sum_{k \in I_{u}\cap I_{v}} (r_{uk}- \mu_{u})^2} \cdot \sqrt{\sum_{k \in I_{u}\cap I_{v}} (r_{vk}- \mu_{v})^2}} $

In [178]:
def pearson_corr(table, x: int, y: int):
    # x and y are the rows of table on which to perform computation
    # get I_x and I_y
    I_x = np.where(np.isnan(table[x]) == False)[0]
    I_y = np.where(np.isnan(table[y]) == False)[0]
    # get the common indices
    I = np.intersect1d(I_x, I_y)
    # get the mean of x and y
    x_mean = np.mean(table[x][I])
    y_mean = np.mean(table[y][I])
    # get the numerator
    numerator = np.sum((table[x][I] - x_mean)*(table[y][I] - y_mean))
    # get the denominator
    denominator = np.sqrt(np.sum((table[x][I] - x_mean)**2)*np.sum((table[y][I] - y_mean)**2))
    # get the correlation
    corr = numerator/denominator
    return corr

Let's compute the correlation coefficient for rows 0 and 2 (1 and 3 in the book). In the book, it is given as 0.894.

In [179]:
pearson_corr(table_2_1, 0, 2)

np.float64(0.8944271909999159)

While we're at it, we can also compute the entire correlation matrix:

In [180]:
# generate a correlation matrix for the table
corr_matrix = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        corr_matrix[i,j] = pearson_corr(table_2_1, i, j)

corr_matrix

array([[ 1.        ,  0.72347804,  0.89442719, -0.8992288 , -0.82422559],
       [ 0.72347804,  1.        ,  0.97072534, -0.72057669, -0.8992288 ],
       [ 0.89442719,  0.97072534,  1.        , -1.        , -0.8660254 ],
       [-0.8992288 , -0.72057669, -1.        ,  1.        ,  0.87705802],
       [-0.82422559, -0.8992288 , -0.8660254 ,  0.87705802,  1.        ]])

Let's write a function that does this:

In [181]:
def corr_matrix_func(table, dimension):
    if dimension == 0:
        number = table.shape[dimension]
        corr_matrix = np.zeros((number, number))
        for i in range(number):
            for j in range(number):
                corr_matrix[i,j] = pearson_corr(table, i, j)
    elif dimension == 1:
        number = table.shape[dimension]
        corr_matrix = np.zeros((number, number))
        table = table.T
        for i in range(number):
            for j in range(number):
                corr_matrix[i,j] = pearson_corr(table, i, j)
    else:
        print("Invalid dimension")
        return None
    return corr_matrix

In [182]:
corr_matrix_func(table = table_2_1, dimension = 0)

array([[ 1.        ,  0.72347804,  0.89442719, -0.8992288 , -0.82422559],
       [ 0.72347804,  1.        ,  0.97072534, -0.72057669, -0.8992288 ],
       [ 0.89442719,  0.97072534,  1.        , -1.        , -0.8660254 ],
       [-0.8992288 , -0.72057669, -1.        ,  1.        ,  0.87705802],
       [-0.82422559, -0.8992288 , -0.8660254 ,  0.87705802,  1.        ]])

In [183]:
corr_matrix_func(table_2_1, 0), corr_matrix_func(table_2_1, 1)

C:\Users\maggo\AppData\Local\Temp\ipykernel_4888\2376229201.py:16: RuntimeWarning: invalid value encountered in scalar divide
  corr = numerator/denominator


(array([[ 1.        ,  0.72347804,  0.89442719, -0.8992288 , -0.82422559],
        [ 0.72347804,  1.        ,  0.97072534, -0.72057669, -0.8992288 ],
        [ 0.89442719,  0.97072534,  1.        , -1.        , -0.8660254 ],
        [-0.8992288 , -0.72057669, -1.        ,  1.        ,  0.87705802],
        [-0.82422559, -0.8992288 , -0.8660254 ,  0.87705802,  1.        ]]),
 array([[1.        , 0.94063416, 0.98782916, 0.89714996, 0.67675297,
         0.57263713],
        [0.94063416, 1.        , 0.99862543, 0.69310328, 0.51449576,
                nan],
        [0.98782916, 0.99862543, 1.        , 0.6381449 , 0.62092042,
         0.62861856],
        [0.89714996, 0.69310328, 0.6381449 , 1.        , 0.81348922,
         0.87038828],
        [0.67675297, 0.51449576, 0.62092042, 0.81348922, 1.        ,
         0.33333333],
        [0.57263713,        nan, 0.62861856, 0.87038828, 0.33333333,
         1.        ]]))

This way we can compute the correlation coefficient both for rows or columns.

Next, we can demean the rating in every line.

In [184]:
def demean_func(table):
    # demean the table
    new_table = np.zeros(table.shape)
    for i in range(table.shape[0]):
        I = np.where(np.isnan(table[i]) == False)[0]
        new_table[i][I] = table[i][I] - np.mean(table[i][I])
    return new_table

In [185]:
table_2_2 = demean_func(table_2_1)
table_2_2

array([[ 1.5,  0.5,  1.5, -1.5, -0.5, -1.5],
       [ 1.2,  2.2,  0. , -0.8, -1.8, -0.8],
       [ 0. ,  1. ,  1. , -1. , -1. ,  0. ],
       [-1.5, -0.5, -0.5,  0.5,  0.5,  1.5],
       [-1. ,  0. , -1. ,  0. ,  1. ,  1. ]])

We also want to store the means somewhere. Let us write a function for that.

In [186]:
def mean_compute(table, dimension = 0):
    # compute the mean of the table
    if dimension == 1:
        table_copy = table.T
    else:
        table_copy = table
    mean = np.zeros(table_copy.shape[0])
    for i in range(table_copy.shape[0]):
        I = np.where(np.isnan(table_copy[i]) == False)[0]
        mean[i] = np.mean(table_copy[i][I])
    return mean

user_mean = mean_compute(table_2_1, 0)
user_mean

array([5.5, 4.8, 2. , 2.5, 2. ])

In [187]:
mean_compute(table_2_1, 1)

array([3.75, 4.5 , 3.25, 2.8 , 3.  , 3.75])

We can use the demeaned table to better predict a rating. Suppose that we want to predict for user in 3 (row-index 2) their ratings for movies 1 and 6 (indices 0 and 5, respectively). We can first compute the set of relevant neighbours as those whose correlation exceeds 0.7 (as an arbitrary benchmark).

In [188]:
corr_matrix_table_2_2 = corr_matrix_func(table_2_1, 0)
corr_matrix_table_2_2

array([[ 1.        ,  0.72347804,  0.89442719, -0.8992288 , -0.82422559],
       [ 0.72347804,  1.        ,  0.97072534, -0.72057669, -0.8992288 ],
       [ 0.89442719,  0.97072534,  1.        , -1.        , -0.8660254 ],
       [-0.8992288 , -0.72057669, -1.        ,  1.        ,  0.87705802],
       [-0.82422559, -0.8992288 , -0.8660254 ,  0.87705802,  1.        ]])

Aside from themselves, this is the case for users 1 and 2 (indices 0 and 1):

In [189]:
k = np.where((corr_matrix_table_2_2[2] > 0.7))
k = k[0][:-1]
k

array([0, 1])

In [190]:
# create a deep copy of table_2_2
table_2_1_copy = table_2_1.copy()
table_2_1_copy[2,0] = user_mean[2] + np.sum(corr_matrix_table_2_2[2][k]*table_2_2[k,0])/np.sum(np.abs(corr_matrix_table_2_2[2][k]))
table_2_1_copy

array([[7.        , 6.        , 7.        , 4.        , 5.        ,
        4.        ],
       [6.        , 7.        ,        nan, 4.        , 3.        ,
        4.        ],
       [3.34386392, 3.        , 3.        , 1.        , 1.        ,
               nan],
       [1.        , 2.        , 2.        , 3.        , 3.        ,
        4.        ],
       [1.        ,        nan, 1.        , 2.        , 3.        ,
        3.        ]])

Let us encapsulate this in a function. We will have to specify a correlation-threshold to determine k, and then within the function we create a deep-copy that fills in the NaN-values.

In [191]:
def imputation_function(table, threshold, dimension):
    table_copy = table.copy()
    # compute correlation_matrix
    corr_matrix = corr_matrix_func(table, dimension)
    # compute means
    user_mean = mean_compute(table, dimension)
    # demean the table
    table_copy_dm = demean_func(table_copy)
    # for each row in correlation matrix compute the k based on threshold
    k = [[] for _ in range(corr_matrix.shape[0])]
    for i in range(corr_matrix.shape[0]):
        k[i] = np.where((corr_matrix[i] > threshold))[0]
    # for each value of k, remove the same number if it is in k, e.g. remove 2 from k[2]
    for i in range(corr_matrix.shape[0]):
        k[i] = np.setdiff1d(k[i], i)
    # for each row and column in the table, compute the imputation
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            if np.isnan(table[i,j]):
                table_copy[i,j] = user_mean[i] + np.sum(corr_matrix[i][k[i]]*table_copy_dm[k[i],j])/np.sum(np.abs(corr_matrix[i][k[i]]))
    return table_copy, corr_matrix, user_mean, k

In [192]:
table_2_1_imputed, corr_matrix_comp, user_mean_comp, k_comp = imputation_function(table_2_1, 0.7, 0)

In [193]:
table_2_1_imputed

array([[7.        , 6.        , 7.        , 4.        , 5.        ,
        4.        ],
       [6.        , 7.        , 6.0135157 , 4.        , 3.        ,
        4.        ],
       [3.34386392, 3.        , 3.        , 1.        , 1.        ,
        0.86431752],
       [1.        , 2.        , 2.        , 3.        , 3.        ,
        4.        ],
       [1.        , 1.5       , 1.        , 2.        , 3.        ,
        3.        ]])

In [194]:
corr_matrix_comp

array([[ 1.        ,  0.72347804,  0.89442719, -0.8992288 , -0.82422559],
       [ 0.72347804,  1.        ,  0.97072534, -0.72057669, -0.8992288 ],
       [ 0.89442719,  0.97072534,  1.        , -1.        , -0.8660254 ],
       [-0.8992288 , -0.72057669, -1.        ,  1.        ,  0.87705802],
       [-0.82422559, -0.8992288 , -0.8660254 ,  0.87705802,  1.        ]])

In [195]:
user_mean_comp

array([5.5, 4.8, 2. , 2.5, 2. ])

In [196]:
k_comp

[array([1, 2]), array([0, 2]), array([0, 1]), array([4]), array([3])]

# Interestingly enough, we can do the entire thing now as well for items!

In [197]:
table_2_1_imputed_cols, corr_matrix_comp_cols, user_mean_comp_cols, k_comp_cols = imputation_function(table_2_1, 0.7, 1)
table_2_1_imputed_cols

C:\Users\maggo\AppData\Local\Temp\ipykernel_4888\2376229201.py:16: RuntimeWarning: invalid value encountered in scalar divide
  corr = numerator/denominator


array([[7.        , 6.        , 7.        , 4.        , 5.        ,
        4.        ],
       [6.        , 7.        , 5.74252405, 4.        , 3.        ,
        4.        ],
       [4.59918476, 3.        , 3.        , 1.        , 1.        ,
        2.10190223],
       [1.        , 2.        , 2.        , 3.        , 3.        ,
        4.        ],
       [1.        , 2.5       , 1.        , 2.        , 3.        ,
        3.        ]])

In [198]:
corr_matrix_comp_cols

array([[1.        , 0.94063416, 0.98782916, 0.89714996, 0.67675297,
        0.57263713],
       [0.94063416, 1.        , 0.99862543, 0.69310328, 0.51449576,
               nan],
       [0.98782916, 0.99862543, 1.        , 0.6381449 , 0.62092042,
        0.62861856],
       [0.89714996, 0.69310328, 0.6381449 , 1.        , 0.81348922,
        0.87038828],
       [0.67675297, 0.51449576, 0.62092042, 0.81348922, 1.        ,
        0.33333333],
       [0.57263713,        nan, 0.62861856, 0.87038828, 0.33333333,
        1.        ]])

In [199]:
k_comp_cols

[array([1, 2, 3]),
 array([0, 2]),
 array([0, 1]),
 array([0, 4, 5]),
 array([3]),
 array([3])]

#### 2.3.1.1 Similarity Function Variants

We have now computed using the pearson-correlation coefficient. Next up, let's turn towards using the cosine-distance as well as z-scores.

Raw Cosine is computed as:

$  RawCosine(u,v) = \frac{\sum_{k \in I_{u} \cap I_{v}} r_{uk}\cdot r_{uk} }{\sqrt{\sum_{k \in I_{u}\cap I_{v}} r_{uk}^2} \cdot \sqrt{\sum_{k \in I_{u}\cap I_{v}} r_{vk}^2}} $

This considerably simplifies computation. In some implementations, the normalization factors do not constrain on k being in the sets of both users but only one in order to make things feasible:

$  FeasibleRawCosine(u,v) = \frac{\sum_{k \in I_{u} \cap I_{v}} r_{uk}\cdot r_{uk} }{\sqrt{\sum_{k \in I_{u}} r_{uk}^2} \cdot \sqrt{\sum_{k \in I_{v}} r_{vk}^2}} $


In [200]:
def RawCosine(table, x: int, y: int):
    # x and y are the rows of table on which to perform computation
    # get I_x and I_y
    I_x = np.where(np.isnan(table[x]) == False)[0]
    I_y = np.where(np.isnan(table[y]) == False)[0]
    # get the common indices
    I = np.intersect1d(I_x, I_y)
    # get the numerator
    numerator = np.sum(table[x][I]*table[y][I])
    # get the denominator
    denominator = np.sqrt(np.sum(table[x][I]**2)*np.sum(table[y][I]**2))
    # get the correlation
    raw_cosine = numerator/denominator
    return raw_cosine

def FeasibleRawCosine(table, x: int, y: int):
    # x and y are the rows of table on which to perform computation
    # get I_x and I_y
    I_x = np.where(np.isnan(table[x]) == False)[0]
    I_y = np.where(np.isnan(table[y]) == False)[0]
    # get the common indices
    I = np.intersect1d(I_x, I_y)
    # get the numerator
    numerator = np.sum(table[x][I]*table[y][I])
    # get the denominator
    denominator = np.sqrt(np.sum(table[x][I_x]**2)*np.sum(table[y][I_y]**2))
    # get the correlation
    feasible_raw_cosine = numerator/denominator
    return feasible_raw_cosine

In [201]:
RawCosine(table_2_1, 0, 2), FeasibleRawCosine(table_2_1, 0, 2)

(np.float64(0.956182887467515), np.float64(0.7766217620286883))

Often we find the Pearson-Correlation to be the preferrable measure as it provides bias-adjustment due to the mean-centering (comp. p. 37).
Inference is impacted by the number of common ratings, $ | I_u \cap I_v | $ between two users u and v. If two users have only few ratings in common, we might want to downweight the impact that this user-pair has on the final imputation. We can do this via significance weighting:

$ DiscountedSim(u,v) = Sim(u,v) * \frac{ \min\{|I_u \cap I_v|, \beta\}}{\beta} $

In [202]:
def DiscountedSim(table, x: int, y: int, beta: float):
    # x and y are the rows of table on which to perform computation
    # get I_x and I_y
    I_x = np.where(np.isnan(table[x]) == False)[0]
    I_y = np.where(np.isnan(table[y]) == False)[0]
    # get the common indices
    I = np.intersect1d(I_x, I_y)
    discounted_similarity = pearson_corr(table, x, y) * np.min([1, len(I)/beta])
    return discounted_similarity

In [203]:
DiscountedSim(table_2_1, 0, 2, 2)

np.float64(0.8944271909999159)

#### 2.3.1.2 Variants of the Prediction Function

We can also use the standard deviation to effectively obtain a (residualized) regression function that uses the z-Scores to predict a user's rating. Effectively, we turn every observed rating into a z-score (demeaned and standardized by SD) of a user, then use for prediction the mean and SD of a user we already know and then use the correlation-function together with the z-score of the same item for other similar users, downweighting by the sum of absolute values of the pearson correlation coefficients:

$ \hat{r}_{uj} = \mu_u + \sigma_u \frac{\sum_{\nu \in P_{u}(j)} Sim(u,v) \cdot z_{\nu j}}{\sum_{\nu \in P_{u}(j)} |Sim(u,v)|} $

In [204]:
def sd_function(table, x: int):
    # x is the row of table on which to perform computation
    # get I_x
    I_x = np.where(np.isnan(table[x]) == False)[0]
    # get the mean of x
    x_mean = np.mean(table[x][I_x])
    # get the standard deviation
    sd = np.sqrt(np.sum((table[x][I_x] - x_mean)**2)/(len(I_x)-1))
    return sd

In [205]:
def sd_vector(table):
    # compute the standard deviation for each row
    sd_vector = np.zeros(table.shape[0])
    for i in range(table.shape[0]):
        sd_vector[i] = sd_function(table, i)
    return sd_vector

In [206]:
def normalize_table(table):
    table_copy = table.copy()
    for i in range(table.shape[0]):
        I = np.where(np.isnan(table[i]) == False)[0]
        table_copy[i][I] = (table[i][I] - np.mean(table[i][I]))/sd_function(table, i)
    return table_copy

In [207]:
def z_score_imputation(table, threshold, dimension: int = 0):
    # x is the row to be imputed
    # y is the list of rows to be used for imputation
    # create a normalized copy of the table
    table_copy = table.copy()
    normalized_table = normalize_table(table)
    # compute user sd
    sd_vector_store = sd_vector(table)
    # compute correlation matrix
    corr_matrix = corr_matrix_func(table, dimension)
    # compute means
    user_mean = mean_compute(table, dimension)
    # for each row in correlation matrix compute the k based on threshold
    k = [[] for _ in range(corr_matrix.shape[0])]
    for i in range(corr_matrix.shape[0]):
        k[i] = np.where((corr_matrix[i] > threshold))[0]
    # for each value of k, remove the same number if it is in k, e.g. remove 2 from k[2]
    for i in range(corr_matrix.shape[0]):
        k[i] = np.setdiff1d(k[i], i)
    # for each row and column in the table, compute the imputation
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            if np.isnan(table[i,j]):
                table_copy[i,j] = user_mean[i] + sd_vector_store[i]*np.sum(corr_matrix[i][k[i]]*normalized_table[k[i],j])/np.sum(np.abs(corr_matrix[i][k[i]]))
    return table_copy, corr_matrix, user_mean, k

In [208]:
table_copy, corr_mat, user_means, k_imputed = z_score_imputation(table_2_1, 0.7, 0)

In [209]:
table_copy

array([[7.        , 6.        , 7.        , 4.        , 5.        ,
        4.        ],
       [6.        , 7.        , 6.37893144, 4.        , 3.        ,
        4.        ],
       [3.04146466, 3.        , 3.        , 1.        , 1.        ,
        1.10483034],
       [1.        , 2.        , 2.        , 3.        , 3.        ,
        4.        ],
       [1.        , 1.52326871, 1.        , 2.        , 3.        ,
        3.        ]])

#### 2.3.1.4 Impact of the long tail

As discussed in the book, similarity weighting can be beneficial if, for example, some items are more often rated than others. For example, we might multiply by the logarithm of the ratio of number of all users to number of users who rated a specific item:

$w_k = \log\Big(\frac{m}{m_k}\Big)$

$ WeightedPearson(u,v) = \frac{\sum_{k \in I_{u} \cap I_{v}} w_k \cdot (r_{uk}- \mu_{u}) \cdot (r_{uk}- \mu_{u}) }{\sqrt{\sum_{k \in I_{u}\cap I_{v}} w_k \cdot (r_{uk}- \mu_{u})^2} \cdot \sqrt{\sum_{k \in I_{u}\cap I_{v}} w_k \cdot (r_{vk}- \mu_{v})^2}} $

In [210]:
# weighted pearson-correlation-matrix
def weighted_pearson(table_input, x: int, y:int, dimension: int = 0):
    table = table_input.copy()
    if dimension == 1:
        table = table.T
    # compute the correlation matrix
    m = table.shape[0]
    weights_pre = [len(np.where(np.isnan(table.T[i]) == False)[0]) for i in range(table.shape[1])]
    weights = np.array([np.log(m/weights_pre[i]) for i in range(table.shape[1])])
    I_x = np.where(np.isnan(table[x]) == False)[0]
    I_y = np.where(np.isnan(table[y]) == False)[0]
    # get the common indices
    I = np.intersect1d(I_x, I_y)
    # get the mean of x and y
    x_mean = np.mean(table[x][I])
    y_mean = np.mean(table[y][I])
    # get the numerator
    numerator = np.sum(weights[I]*(table[x][I] - x_mean)*(table[y][I] - y_mean))
    # get the denominator
    denominator = np.sqrt(np.sum(weights[I] * (table[x][I] - x_mean)**2)*np.sum(weights[I]*(table[y][I] - y_mean)**2))
    # get the correlation
    corr = numerator/denominator
    return corr
    

In [211]:
table_2_1

array([[ 7.,  6.,  7.,  4.,  5.,  4.],
       [ 6.,  7., nan,  4.,  3.,  4.],
       [nan,  3.,  3.,  1.,  1., nan],
       [ 1.,  2.,  2.,  3.,  3.,  4.],
       [ 1., nan,  1.,  2.,  3.,  3.]])

In [212]:
weighted_pearson(table_2_1, 0, 3)

np.float64(-0.9296696802013684)

In [213]:
def weighted_pearson_corr_mat(table, dimension: int = 0):
    if dimension == 0:
        number = table.shape[dimension]
        corr_matrix = np.zeros((number, number))
        for i in range(number):
            for j in range(number):
                corr_matrix[i,j] = weighted_pearson(table, i, j)
    elif dimension == 1:
        number = table.shape[dimension]
        corr_matrix = np.zeros((number, number))
        table = table.T
        for i in range(number):
            for j in range(number):
                corr_matrix[i,j] = weighted_pearson(table, i, j)
    else:
        print("Invalid dimension")
        return None
    return corr_matrix

In [214]:
table_2_1.shape

(5, 6)

In [215]:
weighted_pearson_mat = weighted_pearson_corr_mat(table_2_1, 0)
weighted_pearson_mat

array([[ 1.        ,  0.80428683,  0.89442719, -0.92966968, -0.99811498],
       [ 0.80428683,  1.        ,  1.        , -0.75028028, -0.92163538],
       [ 0.89442719,  1.        ,  1.        , -1.        , -1.        ],
       [-0.92966968, -0.75028028, -1.        ,  1.        ,  0.94087507],
       [-0.99811498, -0.92163538, -1.        ,  0.94087507,  1.        ]])

## 2.3.2 Item-Based Neighbourhood Models

As described in the book, all of the previously given models were user-centric, i.e. we compared two users in their ratings over all products. For this, we computed the set of relevant products for each user-pair and weighted the rating of another user with an appropriate similarity-measure, e.g. the Pearson-Correlation-Coefficient or Cosine-Similarity.
For Item-Based-Methods, we go the other way: we compare items in their similarity and then multiply the ratings of *the same* user for other items multiplied with their similarity for an item at hand. That means if we want to predict how much a user will enjoy a particular brand of car tires, we first compute how similar other items, e.g. windshield wipers, motors, car seats, are to tires, and then predict the user's rating for these car tires as a weighted sum of his ratings for all these other items. 
In contrast, with user-based methods we weight the ratings of *other users* for the *same item* with the similarity between the two *users*. 

$ AdjustedCosine(i,j) = \frac{\sum_{u \in U_i \cap U_j} s_{ui} \cdot s_{uj}}{\sqrt{\sum_{u \in U_i \cap U_j} s_{ui}^2} \cdot \sqrt{\sum_{u \in U_i \cap U_j} s_{ui}^2}} $

where $s_{uj}$ is the demeaned rating of user $u$ for item $j$.

In [216]:
table_2_1

array([[ 7.,  6.,  7.,  4.,  5.,  4.],
       [ 6.,  7., nan,  4.,  3.,  4.],
       [nan,  3.,  3.,  1.,  1., nan],
       [ 1.,  2.,  2.,  3.,  3.,  4.],
       [ 1., nan,  1.,  2.,  3.,  3.]])

In [217]:
i = 0
np.where(np.isnan(table_2_1[:,5]) == False)[0]

array([0, 1, 3, 4])

In [218]:
def AdjustedCosine(table, i: int, j: int):
    # i and j are the columns of table on which to perform computation
    # get U_i and U_j
    U_i = np.where(np.isnan(table[:,i]) == False)[0]
    U_j = np.where(np.isnan(table[:,j]) == False)[0]
    # get the common indices
    U = np.intersect1d(U_i, U_j)
    # get the mean of x and y
    i_mean = np.mean(table[U,i])
    j_mean = np.mean(table[U,j])
    # get the numerator
    numerator = np.sum((table[U,i] - i_mean)*(table[U,j] - j_mean))
    # get the denominator
    denominator = np.sqrt(np.sum((table[U,i] - i_mean)**2)*np.sum((table[U,j] - j_mean)**2))
    # get the correlation
    corr = numerator/denominator
    return corr

In [219]:
AdjustedCosine(table_2_1, 0, 3)

np.float64(0.8971499589146108)

Having computed the adjusted Cosine, let's use it to predict ratings. In order to do that we need to get the top-k-similar items. We can set k as we wish.

In [220]:
def TopKSimilarities(table, u: int, t: int, k: int = 3):
    # compute the top-k adjusted Cosine similarities for user u and item t:
    # get the number of items
    m = table.shape[1]
    # get the similarities
    similarities = np.zeros(m)
    for i in range(m):
        similarities[i] = AdjustedCosine(table, t, i)
    # eliminate the similarity of t with itself
    similarities[t] = 0
    # get the top-k similarities
    top_k = np.argsort(similarities)[-k:]
    return top_k, similarities[top_k]

TopKSimilarities(table_2_1, 0, 3, 3)

(array([4, 5, 0]), array([0.81348922, 0.87038828, 0.89714996]))

In [221]:
table_2_1

array([[ 7.,  6.,  7.,  4.,  5.,  4.],
       [ 6.,  7., nan,  4.,  3.,  4.],
       [nan,  3.,  3.,  1.,  1., nan],
       [ 1.,  2.,  2.,  3.,  3.,  4.],
       [ 1., nan,  1.,  2.,  3.,  3.]])

In [222]:
def AdjCosPrediction(table, u: int, t: int, k: int = 3, threshold: float = 0.7):
    # compute the prediction for user u and item t using the top-k adjusted Cosine similarities
    top_k, similarities = TopKSimilarities(table, u, t, k)
    # keep only the similarities above the threshold
    top_k = top_k[np.where(similarities > threshold)]
    similarities = similarities[np.where(similarities > threshold)]
    # get the mean of the user
    user_mean = np.mean(table[u][np.where(np.isnan(table[u]) == False)[0]])
    # get the prediction
    prediction = user_mean + np.sum(similarities*(table[u][top_k] - np.mean(table[u][np.where(np.isnan(table[u]) == False)[0]])))/np.sum(np.abs(similarities))
    return prediction

In [223]:
AdjCosPrediction(table_2_1, 0, 3, 3, 0.7)

np.float64(5.357962731499528)

In [224]:
# Create an imputation function that takes as input a table and returns the imputed table where the imputation is done using the top-k adjusted cosine similarity if an entry is missing
def AdjCosImputation(table, k: int = 3, threshold: float = 0.7):
    # create a copy of the table
    table_copy = table.copy()
    # for each row and column in the table, compute the imputation
    for i in range(table.shape[0]):
        for j in range(table.shape[1]):
            if np.isnan(table[i,j]):
                table_copy[i,j] = AdjCosPrediction(table, i, j, k, threshold)
    return table_copy

In [225]:
table_2_1

array([[ 7.,  6.,  7.,  4.,  5.,  4.],
       [ 6.,  7., nan,  4.,  3.,  4.],
       [nan,  3.,  3.,  1.,  1., nan],
       [ 1.,  2.,  2.,  3.,  3.,  4.],
       [ 1., nan,  1.,  2.,  3.,  3.]])

Let us replicate the computation on p. 41:

In [226]:
table_2_1_adj_cosine_imputed_items = AdjCosImputation(table_2_1, 2, 0.7)
table_2_1_adj_cosine_imputed_items

C:\Users\maggo\AppData\Local\Temp\ipykernel_4888\229472857.py:16: RuntimeWarning: invalid value encountered in scalar divide
  corr = numerator/denominator


array([[7.        , 6.        , 7.        , 4.        , 5.        ,
        4.        ],
       [6.        , 7.        , 6.50271747, 4.        , 3.        ,
        4.        ],
       [3.        , 3.        , 3.        , 1.        , 1.        ,
        1.        ],
       [1.        , 2.        , 2.        , 3.        , 3.        ,
        4.        ],
       [1.        , 1.        , 1.        , 2.        , 3.        ,
        3.        ]])